<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/simclr-refactor/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision

In [2]:
!pip install gdown

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9681 sha256=4854ece904d628827a034d750cf854efcf1a00aa112ed41ad4958b6e16570695
  Stored in directory: c:\users\jainb1\appdata\local\pip\cache\wheels\ba\e0\7e\726e872a53f7358b4b96a9975b04e98113b005cd8609a63abc
Successfully built gdown


In [2]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
import resnet_simclr

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


In [4]:
def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.STL10('./data', split='train', download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.STL10('./data', split='test', download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

def get_cifar10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.CIFAR10('./data', train=True, download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.CIFAR10('./data', train=False, download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

In [5]:
with open(os.path.join('./config.yml')) as file:
  config = yaml.load(file)

FileNotFoundError: [Errno 2] No such file or directory: './config.yml'

In [21]:
model = resnet_simclr.ResNetSimCLR(base_model='mobilev2', out_dim=128).to(device)
checkpoint = torch.load('checkpoint_0100.pth.tar', map_location=device)
state_dict = checkpoint['state_dict']




# for k in list(state_dict.keys()):

#   if k.startswith('backbone.'):
#     if k.startswith('backbone') and not k.startswith('backbone.fc'):
#       # remove prefix
#       state_dict[k[len("backbone."):]] = state_dict[k]
#   del state_dict[k]

mobilev2 Backbone Selected



In [22]:
# print(state_dict['backbone.classifier.1.2.bias'].shape)
# print('**************************************************************')
# print(model.state_dict().keys())
model.load_state_dict(state_dict)

print(model)

ResNetSimCLR(
  (backbone): MobileNetV2(
    (features): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=Fa

In [23]:
#Relplace the fc layer with 10 classes instead of 128

model.backbone.classifier[1][2] = torch.nn.Linear(1280,10)

# log = model.load_state_dict(state_dict, strict=False)
# assert log.missing_keys == ['fc.weight', 'fc.bias']

In [25]:

train_loader, test_loader = get_stl10_data_loaders(download=True)
# print("Dataset:", config.dataset_name)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
#     print(name)
    if name not in ['backbone.classifier.1.2.weight', 'backbone.classifier.1.2.bias']:
        param.requires_grad = False
    

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
assert len(parameters) == 2  # fc.weight, fc.bias

In [34]:
# Double check that only head is Trainable
model.to(device);
for name, param in model.named_parameters():
    if param.requires_grad != False:
        print(name)
    

backbone.classifier.1.2.weight
backbone.classifier.1.2.bias


In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [36]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [37]:
epochs = 100
for epoch in range(epochs):
  top1_train_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(train_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)
  top1_accuracy = 0
  top5_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    top1_accuracy += top1[0]
    top5_accuracy += top5[0]
  
  top1_accuracy /= (counter + 1)
  top5_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")

Epoch 0	Top1 Train accuracy 17.1875	Top1 Test accuracy: 29.6142578125	Top5 test acc: 81.64794921875
Epoch 1	Top1 Train accuracy 38.45588302612305	Top1 Test accuracy: 46.10595703125	Top5 test acc: 92.451171875
Epoch 2	Top1 Train accuracy 49.267005920410156	Top1 Test accuracy: 52.5927734375	Top5 test acc: 94.6142578125
Epoch 3	Top1 Train accuracy 53.31112289428711	Top1 Test accuracy: 54.716796875	Top5 test acc: 95.28564453125
Epoch 4	Top1 Train accuracy 55.35271072387695	Top1 Test accuracy: 55.6396484375	Top5 test acc: 95.56640625
Epoch 5	Top1 Train accuracy 56.03400802612305	Top1 Test accuracy: 56.04736328125	Top5 test acc: 95.64697265625
Epoch 6	Top1 Train accuracy 56.89338302612305	Top1 Test accuracy: 56.84814453125	Top5 test acc: 95.75439453125
Epoch 7	Top1 Train accuracy 56.9611701965332	Top1 Test accuracy: 57.2802734375	Top5 test acc: 95.8447265625
Epoch 8	Top1 Train accuracy 57.5080451965332	Top1 Test accuracy: 57.56591796875	Top5 test acc: 95.98876953125
Epoch 9	Top1 Train accura